In [ ]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain
!pip install transformers
!pip install accelerate
!pip install pydot
!pip install graphviz
!pip install -U langchain-community


In [ ]:
!apt-get install -y graphviz
!wget -O plantuml.jar "https://github.com/plantuml/plantuml/releases/latest/download/plantuml.jar"
!pip install plantuml graphviz

In [ ]:
import os
import faiss
import json
import torch
import pydot
import graphviz
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer



# Load Sentence Transformer model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Example UML-related knowledge base
uml_knowledge = [
    "A class diagram shows relationships between classes.",
    "A sequence diagram represents object interactions over time.",
    "In UML, aggregation represents a whole-part relationship."
]

# Convert knowledge base to embeddings
knowledge_vectors = embedding_model.encode(uml_knowledge)

# Create FAISS Index
dimension = knowledge_vectors.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(knowledge_vectors)

# Function to retrieve relevant context
def retrieve_similar_context(query):
    query_vector = embedding_model.encode([query])
    _, indices = faiss_index.search(query_vector, k=2)
    retrieved_context = [uml_knowledge[idx] for idx in indices[0]]
    return retrieved_context

# Test retrieval
query = "How do UML class diagrams work?"
print("Retrieved Context:", retrieve_similar_context(query))


# Load Code Llama (7B version) - Open Source
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_uml_description(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(input_ids, max_length=500, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example: Generate UML description
query = "Create a UML class diagram for a Library System"
context = retrieve_similar_context(query)
final_prompt = f"Context: {context}\n\n{query}"
uml_description = generate_uml_description(final_prompt)
print("Generated UML Description:", uml_description)


# Example UML in PlantUML syntax (Modify based on generated description)

uml_code = """
@startuml
class Book {
  +title: String
  +author: Author
  +publisher: Publisher
  +publicationDate: Date
  +genre: String
  +getTitle()
  +getAuthor()
  +getPublisher()
  +getPublicationDate()
  +getGenre()
  +setTitle()
  +setAuthor()
  +setPublisher()
  +setPublicationDate()
  +setGenre()
}

class Author {
  +name: String
  +dateOfBirth: Date
  +dateOfDeath: Date
  +getName()
  +getDateOfBirth()
  +getDateOfDeath()
  +setName()
  +setDateOfBirth()
  +setDateOfDeath()
}

class Publisher {
  +name: String
  +address: String
  +phoneNumber: String
  +getName()
  +getAddress()
  +getPhoneNumber()
  +setName()
  +setAddress()
  +setPhoneNumber()
}

class Library {
  +name: String
  +address: String
  +phoneNumber: String
  +getName()
  +getAddress()
  +getPhoneNumber()
  +setName()
  +setAddress()
  +setPhoneNumber()
}

class Borrower {
  +name: String
  +address: String
  +phoneNumber: String
  +getName()
  +getAddress()
  +getPhoneNumber()
  +setName()
  +setAddress()
  +setPhoneNumber()
}

class Copy {
  +book: Book
  +borrower: Borrower
  +dateBorrowed: Date
  +dateReturned: Date
  +getBook()
  +getBorrower()
  +getDateBorrowed()
  +getDateReturned()
  +setBook()
  +setBorrower()
  +setDateBorrowed()
  +setDateReturned()
}

' Relationships
Book "1" -- "*" Author : "has many"
Book "1" -- "1" Publisher : "has one"
Publisher "1" -- "*" Book : "has many"
Publisher "1" -- "*" Author : "has many"
Library "1" -- "*" Copy : "has many"
Library "1" -- "*" Borrower : "has many"
Borrower "1" -- "*" Copy : "has many"
Copy "1" -- "1" Book : "belongs to"
Copy "1" -- "1" Borrower : "belongs to"

@enduml
"""

# Save to file
with open("library_system.puml", "w") as f:
    f.write(uml_code)

# Render UML Diagram using Python's plantuml package
from plantuml import PlantUML
server = PlantUML(url="http://www.plantuml.com/plantuml/img/")
uml_image = server.processes_file("library_system.puml")

# Display image
from IPython.display import display, Image
display(Image("library_system.png"))